In [19]:
import random
import re
from typing import List

from langchain.agents import AgentExecutor, Tool
from langchain.agents import create_openai_tools_agent
from langchain.tools.retriever import create_retriever_tool
from langchain.chat_models import ChatOpenAI

In [20]:
import sys
sys.path.append(".")

In [21]:
re.search(r"^(\d+)d(\d+)((\+|-)(\d+))?$", "1d20+2")

<re.Match object; span=(0, 6), match='1d20+2'>

In [22]:
def dice_roll(query: str) -> List[int]:
    results = re.search(r"^(\d+)d(\d+)((\+|-)(\d+))?$", query)
    if not results:
        raise ValueError(f"query {query} is uninterpretable")

    num = results.group(1)
    val = results.group(2)
    modifier = results.group(3)
    if modifier:
        mult = 1 if modifier[0] == "+" else -1
        add = int(modifier[1:]) * mult
    else:
        add = 0

    return [random.randint(1, int(val)) + add for i in range(int(num))]    

tools = [
    Tool(
        name="RollDice",
        func=dice_roll,
        description="call this to get the result of rolling dice.",
    ),
]

In [23]:
stats = '''
Small Humanoid (Goblinoid), Neutral Evil
Armor Class 15 (leather armor, shield)
Hit Points 7 (2d6)
Speed 30 ft.
Roll Initiative! +2
STR
8 (-1)
DEX
14 (+2)
CON
10 (+0)
INT
10 (+0)
WIS
8 (-1)
CHA
8 (-1)
Skills Stealth +6
Senses Darkvision 60 ft., Passive Perception 9
Languages Common, Goblin
Challenge 1/4 (50 XP) Proficiency Bonus +2
NIMBLE ESCAPE
Nimble Escape. The goblin can take the Disengage or Hide action as a bonus action on each
of its turns.
Actions
SCIMITAR
Scimitar. Melee Weapon Attack: +4 to hit, reach 5 ft., one target. Hit: 5 (1d6 + 2 )
slashing damage.
SHORTBOW
Shortbow. Ranged Weapon
'''

In [24]:
with open("benchmarks/character-sheets/character_1.txt", "r") as fh:
    sheet = fh.read()

In [26]:
from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages[0].prompt.template = """
You are an experienced dungeon master for Dungeons & Dragons 5th edition. We are going to play a short encounter.
The encounter consists of a player character using the following sheet: {sheet}

The player is fighting two goblins, Goblin A and Goblin B. Here are the stats for goblins: {stats}
First, both the player and the two goblins will make an Initiative roll. The player will report their
roll and you will apply the modifiers in the character sheet.
You will roll dice on behalf of the goblins. When making a roll, you will a) roll the dice and then b) add any relevant
modifiers. Then explain the outcome, for example "Goblin A rolled a [[value]]. With its modifier of [[modifier]], this gives it an Initiative of [[total]]."

You will use all of these values to organize the order in which the participants take their turns. Report that
outcome in a numbered list, with the highest Initiative appearing first.
"""
# IMPORTANT: PARTICIPANTS MAY ONLY ACT WHEN IT IS THEIR TURN.

#When it is a goblin's turn, you will describe what they attempt to do. When it is the player's turn, you will
#prompt the player to describe his or her actions. YOU WILL NEVER ACT ON BEHALF OF THE PLAYER.\
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key=api_key)

In [27]:
import agents.roll_resolver as rr 

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({
    "input": "I rolled a 5",
    "sheet": sheet,
    "stats": stats,
    "chat_history": [
        # here is my character sheet!
    ]
})



> Entering new AgentExecutor chain...

Invoking: `RollDice` with `1d20+2`


[12]
Invoking: `RollDice` with `1d20+2`


[3]Captain Cura rolled a 5 for Initiative. Goblin A rolled a 12 with its modifier of +2, giving it an Initiative of 14. Goblin B rolled a 3 with its modifier of +2, giving it an Initiative of 5. 

The order of the turns will be:
1. Goblin A (Initiative 14)
2. Captain Cura (Initiative 5)
3. Goblin B (Initiative 5)

> Finished chain.


{'input': 'I rolled a 5',
 'sheet': 'Name: Captain Cura\nRace/Ancestry/Heritage: Firbolg\nClass & Level: Barbarian 1\nBackground: Pirate\nAlignment: Chaotic Neutral\nExperience: 0 xp\n\n\nABILITY SCORES & ABILITIES\n(* includes +2 proficiency bonus; ** includes expertise, if any)\n\nStrength 16\n+3 ... Strength Ability Checks\n+5 ... Strength Saving Throws *\n+5 ... Athletics Skill  *\n480 lbs. ... Maximum Carrying Capacity\n960 lbs. ... Max. Push or Drag (Speed -5 ft.)\n960 lbs. ... Maximum Lift\n\nDexterity 13\n+1 ... Dexterity Ability Checks\n+1 ... Dexterity Saving Throws\n+1 ... Acrobatics Skill\n+1 ... Sleight of Hand Skill\n+1 ... Stealth Skill\n\n\nConstitution 14\n+2 ... Constitution Ability Checks\n+4 ... Constitution Saving Throws *\n\nIntelligence 10\n+0 ... Intelligence Ability Checks\n+0 ... Intelligence Saving Throws\n+0 ... Arcana Skill\n+0 ... History Skill\n+0 ... Investigation Skill\n+2 ... Nature Skill  *\n+0 ... Religion Skill\n\nWisdom 14\n+2 ... Wisdom Ability Ch